# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [66]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [67]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [68]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list[0])

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [69]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [70]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [71]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity
    WITH REPLICATION =
    {'class': 'SimpleStrategy','replication_factor':1}
    """)

except Exception as e:
    print(e)
        

#### Set Keyspace

In [72]:
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [73]:
## Define the table for the first query: individual_song_by_session_and_session_item
query1_create_table = "CREATE TABLE IF NOT EXISTS individual_song_by_session_and_session_item "
query1_create_table = query1_create_table + "(session_id int, item_in_session int, artist text,song text, song_length decimal, \
                                            PRIMARY KEY (session_id, item_in_session ))"

##PRIMARY KEY defined as session_id and item_in_session to provide a definition of a unique row within this subset of data.  
##No further cluster columns are requred as no sort order is to be expected from a one-row results set 

#create the individual_songs_by_session_and_session_item table
try:
    session.execute(query1_create_table)
except Exception as e:
    print(e)
      

In [74]:
#import csv data into the individual_song_by_session_and_session_item table, using a loop to insert each relevant item for each field, with explicit data conversion
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        insert_query1 = "INSERT INTO individual_song_by_session_and_session_item (session_id, item_in_session, artist, song, song_length)"
        insert_query1 = insert_query1 + "VALUES (%s,%s,%s,%s,%s);"
        session.execute(insert_query1, (int(line[8]), int(line[3]),line[0],line[9],float(line[5])))
   

#### Do a SELECT to verify that the data have been inserted into each table

In [88]:
#define the query
query1 = "SELECT artist, song, song_length FROM individual_song_by_session_and_session_item WHERE session_id = 338 and item_in_session = 4"

#execute the query
try:
    rows = session.execute(query1)
except Exception as e:
    print(e)

#print resultset to the console and insert values into a pandas dataframe
for row in rows:
    print(row.artist, row.song, row.song_length)
    df1 = pd.DataFrame(data={'artist':[row.artist], 'song':[row.song], 'song_length':[row.song_length]}, columns= ['artist','song','song_length'])

print(df1)

Faithless Music Matters (Mark Knight Dub) 495.3073
      artist                             song song_length
0  Faithless  Music Matters (Mark Knight Dub)    495.3073


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

In [87]:
## Define the table for the second query: artist_and_songs_played_by_named_users
query2_create_table = "CREATE TABLE IF NOT EXISTS artist_and_songs_played_by_named_users "
query2_create_table = query2_create_table + "(user_id int,session_id int, song text,item_in_session_id int, artist text, first_name text, last_name text,  \
                                            PRIMARY KEY ((user_id,session_id),item_in_session_id));"

##PRIMARY KEY defined as 'user_id' and 'session_id' to provide a definition of a unique row within this subset of data.  
##A clustering column has been defined with 'item_in_session_id' to provide an ascending sort order in the returned dataset

#create the individual_songs_by_session_and_session_item table
try:
    session.execute(query2_create_table)
except Exception as e:
    print(e)
    
#import csv data into the artist_and_songs_played_by_named_users table, using a loop to insert each relevant item for each field, with explicit data conversion
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        insert_query2 = "INSERT INTO artist_and_songs_played_by_named_users (user_id,session_id, song, item_in_session_id,artist, first_name, last_name)"
        insert_query2 = insert_query2 + "VALUES (%s,%s,%s,%s,%s,%s,%s);"
        session.execute(insert_query2, (int(line[10]), int(line[8]),line[9],int(line[3]),line[0],line[1],line[4] ))

#define the query
query2 = "SELECT artist, song, user_id FROM artist_and_songs_played_by_named_users WHERE user_id = 10 and session_id = 182"

#execute the query
try:
    rows = session.execute(query2)
except Exception as e:
    print(e)

#print resultset to the console and convert to a pandas dataframe.
holding_list = []
for row in rows:
    print(row.artist, row.song, row.song, row.user_id )      
    holding_list.append([row.artist, row.song, row.user_id])

df2 = pd.DataFrame(holding_list, columns=['artist','song', 'user_id'])
print(df2)

Down To The Bone Keep On Keepin' On Keep On Keepin' On 10
Three Drives Greece 2000 Greece 2000 10
Sebastien Tellier Kilometer Kilometer 10
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Catch You Baby (Steve Pitron & Max Sanna Radio Edit) 10
              artist                                               song  \
0   Down To The Bone                                 Keep On Keepin' On   
1       Three Drives                                        Greece 2000   
2  Sebastien Tellier                                          Kilometer   
3      Lonnie Gordon  Catch You Baby (Steve Pitron & Max Sanna Radio...   

   user_id  
0       10  
1       10  
2       10  
3       10  


In [89]:
## Define the table for the third query: songs_played_by_users
query3_create_table = "CREATE TABLE IF NOT EXISTS song_played_by_users "
query3_create_table = query3_create_table + "(song text, user_id int, item_in_session int, first_name text, last_name text,PRIMARY KEY (song, user_id, item_in_session));"


##PRIMARY KEY defined as the where item 'songs', and user_id and item_in_session to create a definition of a unique row

#create songs_played_by_users table
try:
    session.execute(query3_create_table)
except Exception as e:
    print(e)


#import csv data into the song_played_by_users table, using a loop to insert each relevant item for each field, with explicit data conversion
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        insert_query3 = "INSERT INTO song_played_by_users (song, user_id, item_in_session, first_name, last_name)"
        insert_query3 = insert_query3 + "VALUES (%s,%s,%s,%s,%s);"
        session.execute(insert_query3, 
                        (line[9],int(line[10]),int(line[3]),line[1],line[4]) )

#define the query
query3 = "SELECT first_name, last_name from song_played_by_users WHERE song = 'All Hands Against His Own'"

#execute the query
try:
    rows = session.execute(query3)
except Exception as e:
    print(e)

#print resultset to the console and convert to a pandas dataframe.
holding_list = []
for row in rows:
    print(row.first_name, row.last_name)
    holding_list.append([row.first_name, row.last_name])

df3 = pd.DataFrame(holding_list, columns=['first_name','last_name'])
print(df3)

Jacqueline Lynch
Tegan Levine
Sara Johnson
   first_name last_name
0  Jacqueline     Lynch
1       Tegan    Levine
2        Sara   Johnson


### Drop the tables before closing out the sessions

In [62]:
# drop the unnecessary tables
drop_individual_song_by_session_and_session_item = "DROP TABLE individual_song_by_session_and_session_item"
try:
    rows = session.execute(drop_individual_song_by_session_and_session_item)
except Exception as e:
    print(e)
    
    
drop_artist_and_song_by_session_and_user = "DROP TABLE artist_and_songs_played_by_named_users"
try:
    rows = session.execute(drop_artist_and_song_by_session_and_user)
except Exception as e:
    print(e)


drop_song_played_by_users = "DROP TABLE song_played_by_users"
try:
    rows = session.execute(drop_song_played_by_users)
except Exception as e:
    print(e)
    

### Close the session and cluster connection¶

In [63]:
#close down session and connection to cluster
session.shutdown()
cluster.shutdown()